# Predict

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

### Imports

In [ ]:
# Imports
import os
import warnings
import tools
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from matplotlib import pyplot
import numpy as np
from sklearn.metrics import confusion_matrix
import sklearn
import wandb

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Data

In [ ]:
# Root CSV files directory
dirname = "./data/absolute/modified/"  


# Load data and print summary, if desired
x_train, x_val, x_test, y_train, y_val, y_test, labels = tools.load_from(dirname, verbose=True) 

In [ ]:
with open('tokens_json.txt', 'r') as outfile:
    json_ex = outfile.read()
    

tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_ex)
print(tokenizer.word_index)
token_labels = {y:x for x,y in tokenizer.word_index.items()}

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
model = tf.keras.models.load_model('model-best_2d.h5')
model.summary()

### Evaluate

In [ ]:
eval = model.evaluate(x_train, y_train, verbose=2)

In [ ]:
predict = model.predict(x_train)
#print(predict)

y_pred = np.array([np.argmax(pred) for pred in predict])

y_pred_integer = np.argmax(predict, axis=1)
y_train_integer = np.argmax(y_train, axis=1)

print(x_train_integer)

y_pred_name = ([token_labels[p] for p in y_pred_integer])
y_train_name = ([token_labels[p] for p in y_train_integer])


In [ ]:
y_pred_name

In [ ]:
y_test_name 

In [ ]:
wandb.init(project="Test", name="confusion")

In [ ]:
# Confusion Matrix
#sklearn.metrics.plot_confusion_matrix(y_test_name, y_pred_name)
wandb.sklearn.plot_confusion_matrix(y_train_name, y_pred_name)

# TFLITE

In [ ]:
x_test=np.array(x_test,dtype=np.float32)


print(type(x_train[0][0][0]))
print(type(x_test[0][0][0]))


In [ ]:
import numpy as np
import tensorflow as tf
from time import time

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition_3D.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)


input_shape = input_details[0]['shape']
time1= time()
input_data = [1]
predictions = []

input_details[0]["index"]

for x_pred in x_test:
    
    input_data[0] = x_pred
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data))
    
predictions = [token_labels[p] for p in predictions]

In [ ]:
for i in range(0, len(predictions)):
    print('keras: ',y_pred_name[i],  predictions[i])

In [ ]:
print("Predictions:", predictions)
time2= time()
dauer = time2 - time1
print("Anzahl Samples: ", len(predictions))
print("Predictiondauer: ", dauer)
print("Samples pro Sekunde:", (len(predictions)/dauer))

In [ ]:
print(input_data)